# TP 3 crawling
* Robin MASLIAH



> http://www.freepatentsonline.com/result.html?sort=relevance&srch=top&query_txt=video&submit=&patents=on

In [277]:
from bs4 import BeautifulSoup
from json import loads
from urllib.request import urlopen
from urllib.parse import urlencode
from sqlalchemy import create_engine
import pandas as pd
from tidylib import tidy_document
import re
import requests
import urllib
import time
import os
import json

In [398]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [399]:
print("Le fichier se situe ici : ", os.getcwd())

Le fichier se situe ici :  /home/robin/Documents/P4/Web/TP3


# Partie 1

## Question 1
> wget http://www.freepatentsonline.com/8591332.html
* Récupère le fichier html de la page.

## Question 2
> wget -r --wait=1 http://www.freepatentsonline.com/8591332.html
* Télécharge de manière récursive les pages et les contenus CSS, HTML du site.

## Question 3
> wget -np http://www.freepatentsonline.com/8591332.html
* Ne télécharge que les pages Web contenues dans le même répertoire que la page Web initiale.

## Question 4
> wget -r --wait=1 -np --accept-regex "([0-9]*.html|[0-9]{6}.html)" http://www.freepatentsonline.com/
* Télécharge les pages contenants des brevets.

# Partie 2

In [206]:
url_link = 'http://www.freepatentsonline.com/result.html?sort=relevance&srch=top&query_txt=video&submit=&patents=on'

## Etape 1
> Récuperer le html pour une page.

In [340]:
def etape1(url_link):
    f = urllib.request.urlopen(url_link).read()
    return f

## Etape 2
> Récupérer html sur 30 pages et le stocker dans une liste.

In [404]:
def etape2(f):
    brevets_html = []
    for i in range(1, 31):
        lien = "http://www.freepatentsonline.com/result.html?" + str(i) + "'sort=relevance&srch=top&query_txt=video&submit=&patents=on"
        brevets_html.append(etape1(lien))
        time.sleep(5)
    return brevets_html

## Etape 3
> Parser les html de toutes les pages et faire une listes de tous les liens présents dans le codes html.

In [342]:
def etape3(f):
    brevets = []
    for j, i in enumerate(f):
        html = f[j]
        soup = BeautifulSoup(html, 'html.parser')
        for a in soup.find_all('a', href=True):
            brevets.append(a.get('href'))
        time.sleep(5)
    return brevets

## Etape 4
> S'assurer qu'il n'y ait point de doublon parmi les liens extraits.

In [343]:
def etape4(f):
    brevets_unique = set(f)
    list_brevets = list(brevets_unique)
    return list_brevets

## Etape 5
> De manière empirique on enlève les liens qui ne semblent pas pointer vers des pages de brevets.

In [344]:
def etape5(f):
    bon_lien = []
    
    for j, i in enumerate(f):
        string = f[j]
        result = re.search('(?<=/)\w+', string)
        if result is not None:
            bon_lien.append(string)
        else:
            pass
    return bon_lien

## Etape 6
> On supprime bien tous les liens parasites qui ne sont pas des pages de brevets.

In [345]:
def etape6(f):
    liste_liens = pd.DataFrame(f)
    liste_liens.columns = ['a']
    engine = create_engine('sqlite:///:memory:')
    liste_liens.to_sql('table', engine)
    pd.read_sql_table('table', engine)
    df = pd.read_sql_query("select * from 'table' where a not like 'http%' and a not like '/se%' and a not like '/cont%' and a not like '/tools%' and a not like '/regist%' and a not like '/privac%' ", con=engine)
    liste_liens = list(df['a'])
    return liste_liens

## Etape 7
> Récupérer informations supplémentaire sur les pages brevets pour les stocker en json.

In [346]:
def etape7(liste_liens):
    brevets = {}
    for j, i in enumerate(liste_liens):
        lien = "http://www.freepatentsonline.com" + str(liste_liens[j])
        html = etape1(lien)
        doc, error = tidy_document(html, options={'numeric-entities': 1})
        soup = BeautifulSoup(html, 'html.parser')
        brevets[lien] = {}
        for info in soup.find_all('form', {'name': 'biblio'})[0].find_all('input'):
            if info.get('name') == 'title':
                brevets[lien]['Title'] = info.get('value')
            elif info.get('name') == 'author':
                brevets[lien]['Author'] = info.get('value')
            elif info.get('name') == 'assignee':
                brevets[lien]['Assignee'] = info.get('value')
            elif info.get('name') == 'country':
                brevets[lien]['Country'] = info.get('value')
            elif info.get('name') == 'patent':
                brevets[lien]['Patent'] = info.get('value')
            elif info.get('name') == 'year':
                brevets[lien]['Year'] = info.get('value')
            elif info.get('name') == 'month':
                brevets[lien]['Month'] = info.get('value')
    time.sleep(5)
    with open('info_patents.json', 'w') as fp:
        json.dump(brevets, fp)
    return brevets

## Etape 8
> Filtre sur les auteurs français.

In [392]:
def etape8(patent_json):
    patent_json = pd.DataFrame(patent_json)
    patent_json = patent_json.T
    engine = create_engine('sqlite:///:memory:')
    patent_json.to_sql('table2', engine)
    pd.read_sql_table('table2', engine)
    df = pd.read_sql_query("select * from 'table2' where 'Author' like '%FR%'", con=engine)
    return df

## Test des fonctions

In [405]:
brevets_html = etape2(url_link)

In [370]:
brevets = etape3(brevets_html)

In [407]:
brevets

['/',
 '/register.html',
 '/search.html',
 '/search.html',
 '/',
 '/search.html',
 'http://research.freepatentsonline.com',
 'http://research.freepatentsonline.com/mpep',
 '/tools-resources.html',
 'http://research.freepatentsonline.com/acclaimip',
 'http://research.freepatentsonline.com/help',
 'result.html?p=2&srch=top&query_txt=video&patents=on',
 'result.html?p=3&srch=top&query_txt=video&patents=on',
 'result.html?p=4&srch=top&query_txt=video&patents=on',
 'result.html?p=5&srch=top&query_txt=video&patents=on',
 'result.html?p=6&srch=top&query_txt=video&patents=on',
 'result.html?p=7&srch=top&query_txt=video&patents=on',
 'result.html?p=8&srch=top&query_txt=video&patents=on',
 'result.html?p=9&srch=top&query_txt=video&patents=on',
 'result.html?p=10&srch=top&query_txt=video&patents=on',
 'result.html?p=11&srch=top&query_txt=video&patents=on',
 'result.html?p=12&srch=top&query_txt=video&patents=on',
 'result.html?p=13&srch=top&query_txt=video&patents=on',
 'result.html?p=14&srch=top&

In [371]:
brevets_unique = etape4(brevets)

In [372]:
liste_liens = etape5(brevets_unique)

In [373]:
liste_liens = etape6(liste_liens)

In [408]:
liste_liens

['/y2007/0203911.html',
 '/EP2466885A2.html',
 '/8917307.html',
 '/y2004/0095396.html',
 '/y2013/0329138.html',
 '/y2003/0156188.html',
 '/y2007/0058080.html',
 '/y2013/0176437.html',
 '/y2010/0134692.html',
 '/EP1349398A1.html',
 '/y2017/0237964.html',
 '/DE4018956C2.html',
 '/7848623.html',
 '/WO2004080050A2.html',
 '/8760566.html',
 '/EP2031824A2.html',
 '/8591332.html',
 '/y2006/0204115.html',
 '/6970083.html',
 '/y2013/0222419.html',
 '/WO2005086471A1.html',
 '/y2016/0351229.html',
 '/y2011/0150090.html',
 '/9357192.html',
 '/y2016/0093331.html',
 '/y2015/0174477.html',
 '/y2014/0337900.html',
 '/y2007/0124766.html',
 '/y2006/0221255.html',
 '/9262753.html',
 '/6025874.html',
 '/9299388.html',
 '/y2017/0070772.html',
 '/WO1997003524A1.html',
 '/y2009/0136205.html',
 '/9350943.html',
 '/9854301.html',
 '/y2017/0310576.html',
 '/y2010/0178038.html',
 '/DE10133583B4.html',
 '/y2004/0143838.html',
 '/8463961.html',
 '/9591319.html',
 '/y2009/0196569.html',
 '/EP3016389A1.html',
 '/458

In [374]:
patent_json = etape7(liste_liens)

In [409]:
patent_json

{'http://www.freepatentsonline.com/4581644.html': {'Assignee': 'RCA Corporation (Princeton, NJ)',
  'Author': 'Deiss, Michael S. (Indianapolis, IN)',
  'Country': 'United States',
  'Month': 'April',
  'Patent': '4581644',
  'Title': 'Video bus',
  'Year': '1986'},
 'http://www.freepatentsonline.com/6025874.html': {'Assignee': 'Ultrak, Inc. (Lewisville, TX)',
  'Author': 'Cooper, Alan Neal (Coppell, TX), Bauerle, David William (Coppell, TX)',
  'Country': 'United States',
  'Month': 'February',
  'Patent': '6025874',
  'Title': 'Video multiplexer',
  'Year': '2000'},
 'http://www.freepatentsonline.com/6970083.html': {'Assignee': 'Objectvideo, Inc. (Reston, VA, US)',
  'Author': 'Venetianer, Peter L. (McLean, VA, US), Brewer, Paul C. (Arlington, VA, US), Chosak, Andrew J. (McLean, VA, US), Clark I, John W. (Leesburg, VA, US), Haering, Niels (Arlington, VA, US), Lipton, Alan J. (Herndon, VA, US), Myers, Gary (Ashburn, VA, US), Yen, Chung-cheng (Clifton, VA, US), Kalapa, Pramod (Vienna, V

In [393]:
french_authors = etape8(patent_json)

In [396]:
french_authors

,index,Assignee,Author,Country,Month,Patent,Title,Year


> Je ne trouve pas d'auteurs français parmi les breves que j'ai scrappé.